## NOTES:
<ul>
<li>Sections labeled <strong>CORE</strong> must be run in order to begin Aquifer Analysis</li>
<li>Cells labeled <strong>Control</strong> contain inputs for the immeadiately proceeding section(s)</li>
<li>Sections labeled <strong>EXTRA</strong> contain additional plotting or analysis tools but are not necessary for Aquifer Analysis</li>
</ul>

## CORE: Imports

In [1]:
#Python3.10
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import contextily as cx
import requests
import calendar
from importlib import reload
from typing import IO

from datetime import datetime, timedelta
from shapely.geometry import Point
from io import StringIO
from mpl_toolkits.axes_grid1 import make_axes_locatable

# USGS Data retreival tool
from dataretrieval import nwis, utils, codes

# Custom modules are imported in multiple locations to faciliate easy reloading when edits are made to their respective files
import Src.classes as cl
import Src.func as fn
reload(cl)
reload(fn)

# TODO: Look into the warning that this is disabling. It doesn't appear to be significant for the purposes of this code but should be understood
pd.options.mode.chained_assignment = None

#pd.options.mode.chained_assignment = 'warn'

c:\Users\alekh\AppData\Local\Programs\Python\Python310\lib\site-packages\dataretrieval\nadp.py:44: UserWarning: GDAL not installed. Some functions will not work.
  warnings.warn('GDAL not installed. Some functions will not work.')


## CORE: Single Site Data<br>
This function produces the streamflow analysis (using the functions above) for a single site given a list of thresholds and time windows to analyze

Controls:

In [2]:
import Src.classes as cl
import Src.func as fn
reload(cl)

# df2 holds all-time data, df is analyzed range
curr_guage = cl.SRB_Guage
df = nwis.get_record(sites='09306200', service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end='2020-09-30')
df2 = nwis.get_record(sites=curr_guage.id, service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end='2014-09-30')

# Set to true if running indepedently to verify data with Kocis paper (or other source)
# Set to false if running Aquifer Analysis
testing = False

Single Site Analysis:

In [3]:
# If looking at a post impairment period, but calculating threshold based on the full record of data, pass a second dataframe with a different 
# start/end date as the final parameter. This method was used in Kocis 2017 and is needed for some data verification, but is not the methodology
# used for the Aquifer Analysis and so *args will most often be empty.
import Src.func as fn
reload(fn)

def single_site_data(df: pd.DataFrame, quantiles_list: list, data_ranges_list: list, *args):
    df = df.reset_index()    
    threshold = None
    df_complete_site_data = pd.DataFrame()    
    df_complete_mk_mag = pd.DataFrame()
    df_complete_mk_dur = pd.DataFrame()
    df_complete_mk_intra = pd.DataFrame()
    
    for data_range in data_ranges_list:
        for quantile in quantiles_list:
            # Copying original dataframe to avoid any conflicts
            df_copy = df.copy()
            
            # Validate that site is not missing > 10% of data
            date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * data_range)              
            
            # Filter dataframe based on current data_range (don't do this during testing if testing unique dataset/range)
            if not testing:
                df_copy = df_copy[df_copy['datetime'].dt.date >= date_threshold]
            
            # Validate <10% missing data based on filtered dataframe
            missing = fn.validate(df_copy, date_threshold, fn.DEFAULT_END)
            valid = missing < fn.MAX_MISSING_THRESHOLD
            missing = int(round(missing, 2) * 100)                               
    
            # Check for optional second dataframe containing full record
            if args and isinstance(args[0], pd.DataFrame) and not args[0].empty:
                #print('Threshold calculation across the full record')
                df2 = args[0].reset_index()
                threshold = fn.calc_threshold(df2, quantile)
                # threshold = 52350 # SRB_Guage post-impairment threshold for verification with Kocis_2017 
            else:
                #print('Threshold calculation across a limited record')
                threshold = fn.calc_threshold(df_copy, quantile)                             

            # Create a dataframe with only days over HMF threshold as well as continuous dataframe for MK trend test
            hmf_series_defl, hmf_series_cont = fn.filter_hmf(df_copy, threshold)
            #print(zero_deflated_hmf)
            
            # Aggregate data before performing MK magnitude test
            df_agg_cont = hmf_series_cont.copy()
            df_agg_cont = fn.convert_hmf(df_agg_cont, threshold)
            df_agg_cont['00060_Mean'] = df_agg_cont['00060_Mean'].apply(lambda x: x * fn.CUBIC_FT_KM_FACTOR if x >= 0 else 0)
            df_agg_cont.set_index('datetime', inplace=True)
            df_agg_cont = df_agg_cont.resample(fn.HYDRO_YEAR).agg({'00060_Mean': ['sum', 'count']})
            df_agg_cont.columns = ['Sum', 'Count']
            df_agg_cont_defl = df_agg_cont[df_agg_cont['Sum'] > 0]          
            
            # MK Magnitude
            df_mk_mag = fn.mann_kendall(df_agg_cont_defl['Sum'], df_agg_cont['Sum'], fn.MK_TREND_ALPHA)

            # Find number of years with HMF
            hmf_years = fn.num_hmf_years(hmf_series_defl)            

            # Mask out months that don't fall within 3 and 6 month Winter range
            df_six_month, df_three_month = fn.three_six_range(hmf_series_defl, 12, 2, 11, 4)

            # Convert to daily average flow in cfpd, and take only flow above the threshold
            hmf_series_defl = fn.convert_hmf(hmf_series_defl, threshold)
            total_hmf_flow = hmf_series_defl["00060_Mean"].sum()
            hmf_per_month = fn.monthly_hmf(hmf_series_defl, data_range, quantile)

            # Calculate 3 and 6 month HMF
            df_six_month = fn.convert_hmf(df_six_month, threshold)
            six_month_hmf = df_six_month["00060_Mean"].sum()
            df_three_month = fn.convert_hmf(df_three_month, threshold)
            three_month_hmf = df_three_month["00060_Mean"].sum()
            
            total_hmf_flow = (total_hmf_flow * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            six_month_hmf = (six_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            three_month_hmf = (three_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years

            # Inter-annual
            inter_annual, delta = fn.calc_inter_annual(hmf_series_cont, hmf_years)            

            # Average Duration and Intra-annual Frequency
            hmf_series_cont = fn.convert_hmf(hmf_series_cont, threshold)
            event_duration, annual_duration, intra_annual, event_hmf, df_results = fn.calc_duration_intra_annual(hmf_series_cont, hmf_years)
            dur_series_defl = df_results['duration'][df_results['duration'] > 0]
            dur_series_cont = df_results['duration']
            df_mk_dur = fn.mann_kendall(dur_series_defl, dur_series_cont, fn.MK_TREND_ALPHA)
                        
            intra_series_defl = df_results['total_events'][df_results['total_events'] > 0]
            intra_series_cont = df_results['total_events']
            df_mk_intra = fn.mann_kendall(intra_series_defl, intra_series_cont, fn.MK_TREND_ALPHA) 
            
            # Timing Calculation using DOHY
            timing = fn.calc_timing(hmf_series_defl)           

            # TODO: One-day peaks (avg. # of times hmf occurs on one day only)          
            
            # Merging site dataframe with Mann-Kendall dataframe. This start date is the beginning of the actual data, not necessarily 
            # the beginning of the analyzed range. Validation (above) starts from the start of the official range (1970/90-2020)
            start = df_copy['datetime'].min().date()
            end = df_copy['datetime'].max().date()
            data = {'dataset_ID': (data_range * quantile), 'site_no': df_copy.iloc[0]['site_no'], 'analyze_start': start, 'analyze_end': end, 'analyze_range': delta, 'quantile': quantile, 
                    'valid': valid, 'missing_data%': missing, 'threshold': threshold, 'hmf_years': hmf_years, 'annual_hmf': total_hmf_flow, 'six_mo_hmf': six_month_hmf, 'three_mo_hmf': three_month_hmf, 
                    'annual_duration': annual_duration, 'event_duration': event_duration, 'event_hmf': event_hmf, 'inter_annual%': inter_annual, 'intra_annual': intra_annual, 'timing': timing}              
            
            # Merging MK magnitiude
            df_mk_mag.insert(0, 'dataset_ID', data_range * quantile)
            df_mk_mag.insert(1, 'site_no', df_copy.iloc[0]['site_no'])
            df_complete_mk_mag = pd.concat([df_complete_mk_mag.reset_index(drop=True), df_mk_mag.reset_index(drop=True)], axis=0) 
            
            # Merging MK duration
            df_mk_dur.insert(0, 'dataset_ID', data_range * quantile)
            df_mk_dur.insert(1, 'site_no', df_copy.iloc[0]['site_no'])
            df_complete_mk_dur = pd.concat([df_complete_mk_dur.reset_index(drop=True), df_mk_dur.reset_index(drop=True)], axis=0)   
            
            # Merging MK intra-annual
            df_mk_intra.insert(0, 'dataset_ID', data_range * quantile)
            df_mk_intra.insert(1, 'site_no', df_copy.iloc[0]['site_no'])
            df_complete_mk_intra = pd.concat([df_complete_mk_intra.reset_index(drop=True), df_mk_intra.reset_index(drop=True)], axis=0)        
            
            # Merging metric results
            df_single_iteration = pd.DataFrame(data, index=['0'])
            df_single_iteration = pd.concat([df_single_iteration.reset_index(drop=True), hmf_per_month.reset_index(drop=True)], axis=1)
            df_complete_site_data = pd.concat([df_complete_site_data.reset_index(drop=True), df_single_iteration.reset_index(drop=True)], axis=0)
        
    return df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra

# For testing purposes, to run this cell independently
df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)

'''
try:
    with pd.ExcelWriter('df_single_site.xlsx') as writer:
        df_complete_site_data.to_excel(writer, sheet_name='site_metrics', index=False)
        df_complete_mk_mag.to_excel(writer, sheet_name='mk_magnitude', index=False)
        df_complete_mk_intra.to_excel(writer, sheet_name='mk_intra', index=False)
        df_complete_mk_dur.to_excel(writer, sheet_name='mk_duration', index=False)
except Exception as e:
    print(e)
'''
   
fn.single_site_report(df_complete_site_data)
df_complete_site_data = fn.gages_2_filtering(df_complete_site_data)
#fn.save_data(df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra, 'TEST')


Site No: 0    09306200
1    09306200
2    09306200
0    09306200
Name: site_no, dtype: object
Analyzed Range: 30
30
50
50
Quantile: 0.90
0.95
0.90
0.95
Valid: True
True
True
True
% Missing: 3
3
2
2
90%: 34.520
53.855
53.000
76.000
HMF Years: 21
14
30
20
Annual Duration: 50.714286
38.071429
59.166667
44.450000
Event Duration: 15.213152
19.197619
18.188333
17.307063
Event HMF: 0.001363
0.003407
0.002810
0.002751
Inter-annual Frequency: 70
47
60
40%
Intra-annual Frequency: 2.761905
2.428571
3.133333
3.150000
Total HMF in km^3/year: 0.005196
0.005295
0.007876
0.008238
Center of Mass: 236.571429
228.142857
214.800000
225.800000
6 Month HMF in km^3/year: 0.001471
0.001448
0.002101
0.002069
3 Month HMF in km^3/year: 0.000025
0.000003
0.000074
0.000006


## CORE: Multi-Site Filtering<br>
This function creates the list of sites to analyze by filtering the complete list of state sites with 00060_Mean data down to those that lie within a specific watershed boundary using decimal long/lat positional data and a region shapefile (e.g. state or watershed boundary)<br><br>

Controls:

In [4]:
# This is only needed when running the following cell independently
shapefile_path = 'ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp'

Site List Generation:

In [5]:
def filter_state_site(shapefile_path: str, state_uri: str):
    """Creates a list of sites with over 50 years of 00060_Mean streamflow data for a given region"""
    # Request page from USGS site, ignore all informational lines
    response = requests.get(state_uri)
    data = response.text
    lines = data.splitlines()
    lines = [line for line in lines if not line.startswith('#')]

    # Create dataframe where site_no is a list of all sites in a state with 00060 data
    tsd = "\n".join(lines)
    df = pd.read_csv(StringIO(tsd), sep='\t')
    df_state_sites = df.iloc[1:]
        
    # Filter out sites outside of HU boundary
    if fn.SORT_BY_WB:
        shapefile = gpd.read_file(shapefile_path)
        df_state_sites['geometry'] = [Point(lon, lat) for lon, lat in zip(df_state_sites['dec_long_va'], df_state_sites['dec_lat_va'])]
        gdf_data = gpd.GeoDataFrame(df_state_sites, crs=shapefile.crs)
        df_state_sites = gpd.sjoin(gdf_data, shapefile, predicate='within')
            
    #print(df_state_sites.columns.to_list())
    #print(df_state_sites)
    
    return df_state_sites

df_state_sites = filter_state_site(shapefile_path, fn.SITES_URI)
print(f'Total Sites: {len(df_state_sites)} in the state of {fn.STATE_CODE.upper()} in the given WB')
site_list = df_state_sites['site_no'].to_list()
print(site_list)

Total Sites: 86 in the state of CA in the given WB
['09429000', '09429100', '09429155', '09429180', '09429200', '09429500', '09429600', '09521100', '09523000', '09523200', '09523400', '09523600', '09523800', '09524000', '09524700', '09525000', '09526200', '09530000', '09530500', '11251000', '11253310', '11255575', '11261100', '11261500', '11262900', '11273400', '11274000', '11274500', '11274550', '11274630', '11289850', '11290000', '11303000', '11303500', '11304810', '11311300', '11312672', '11312676', '11312685', '11312968', '11313240', '11313315', '11313405', '11313431', '11313433', '11313434', '11313440', '11313460', '11336600', '11336685', '11336790', '11336930', '11337080', '11370500', '11370700', '11374000', '11376000', '11376550', '11379500', '11389500', '11390500', '11421000', '11424000', '11425500', '11446500', '11447360', '11447650', '11447830', '11447850', '11447890', '11447903', '11447905', '11451800', '11452500', '11452800', '11452900', '11453000', '11455095', '11455140', 

## CORE: National Metrics Dataset
Generates a nationwide dataset for all sites meeting datarange criteria (lengthy runtime!)

In [6]:
huc2_gdf = gpd.read_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')
huc4_gdf = gpd.read_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
aq_gdf = gpd.read_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

In [7]:
state_limit = 50
dataset_name = 'National_Metrics'
shapefile_path = 'ShapeFiles/Lower48/lower48.shp'

# States with few sites for testing purposes
test_state_list = ['ME', 'DE']

# Set to False to ignore the blacklist and analyze all sites. This could be occasionally
# worth doing as USGS could update site data making it valid in the future
allow_blacklist = True

curr_blacklist = []
try:
    with open('Prelim_Data/_National_Metrics/National_Sites_Blacklist.txt', 'r') as file:
        curr_blacklist = file.read().split(', ')
        curr_blacklist = [x.replace("'", "") for x in curr_blacklist]
except Exception as e:
    print(e)

In [9]:
df_natl_metrics = pd.DataFrame()
df_natl_mk_mag = pd.DataFrame()
df_natl_mk_dur = pd.DataFrame()
df_natl_mk_intra = pd.DataFrame()

natl_blacklist = []

for state_index, state in enumerate(fn.STATE_LIST):
    if state_index >= state_limit: break
    
    print(f'[---Working on {state}---]')
    state_uri = fn.create_state_uri(state, fn.PARAM_CODE)
    df_state_sites = filter_state_site(shapefile_path, state_uri)
    df_state_sites = df_state_sites.reset_index()
    print(f'Total Sites: {len(df_state_sites)} in the state of {state}')
    
    # Modified version of the create_multi_site_data() function
    for site_index, row in df_state_sites.iterrows():
        
        if allow_blacklist and str(row['site_no']) in curr_blacklist:
            print(f'IGNORED: Site {row["site_no"]} is in aquifer blacklist')
            continue
        
        df = nwis.get_record(sites=row['site_no'], service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end=fn.DEFAULT_END)
        df = df.reset_index()
        print(f'Working on {state} site {site_index + 1}/{len(df_state_sites)} ({row["site_no"]})')
        
        if df.empty:
            natl_blacklist.append(row['site_no'])
            print(f'IGNORED: No data for site {row["site_no"]}')
            continue
        
        if '00060_Mean' not in df.columns:
            natl_blacklist.append(row['site_no'])
            print(f'IGNORED: No 00060_Mean data for site {row["site_no"]}')
            continue
        
        start = df['datetime'].min().date()
        end = df['datetime'].max().date()
        range = round((end - start).days / 365.25, 1)
        
        if range < fn.MIN_DATA_PERIOD:
            natl_blacklist.append(row['site_no'])
            print(f'IGNORED: Not enough data for site {row["site_no"]}')
            continue
        
        point = Point(row['dec_long_va'], row['dec_lat_va'])
        #state_code = row['station_nm'].strip()[-2:]
        
        huc2 = huc4 = aquifer = 'NA'
        # HUC2 assignment
        for i, geo_row in huc2_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                huc2 = geo_row['huc2_code']
                continue
            
        # HUC4 assignment
        for i, geo_row in huc4_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                huc4 = geo_row['huc4_code']
                continue
            
        # Aquifer assignment    
        for i, geo_row in aq_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                aquifer = geo_row['aq_name']
                continue        
        
        # A few very broken sites with almost no data can have 0 hmf years and cause errors (i.e. '03592000')
        # so we'll catch these and add them to the site blacklist
        try:
            df_single_site_metric, df_mk_mag, df_mk_dur, df_mk_intra = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)
            add_data = {'dec_lat_va': row['dec_lat_va'], 'dec_long_va': row['dec_long_va'], 'data_start': start, 'data_end': end, 'total_record': range, 
                        'state': state, 'huc2_code': huc2, 'huc4_code': huc4, 'within_aq': aquifer}			
            add_data = pd.DataFrame(add_data, index=['0'])
        except Exception as e:
            natl_blacklist.append(row['site_no'])
            print(f"ERROR: Single site data failure for site {row['site_no']}:\n{e}")
            continue
        
        add_data = pd.DataFrame(np.tile(add_data.values, (len(df_single_site_metric), 1)), columns=add_data.columns)
        df_single_site_metric = pd.concat([df_single_site_metric.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_mag = pd.concat([df_mk_mag.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_dur = pd.concat([df_mk_dur.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_intra = pd.concat([df_mk_intra.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        
        # Append single site data to multi-site dataframes
        df_natl_metrics = pd.concat([df_natl_metrics, df_single_site_metric], ignore_index=True)
        df_natl_mk_mag = pd.concat([df_natl_mk_mag, df_mk_mag], ignore_index=True)
        df_natl_mk_dur = pd.concat([df_natl_mk_dur, df_mk_dur], ignore_index=True)
        df_natl_mk_intra = pd.concat([df_natl_mk_intra, df_mk_intra], ignore_index=True)
        
df_natl_metrics = fn.gages_2_filtering(df_natl_metrics)

# Create national site blacklist
try:
    with open(f'Prelim_Data/National_Sites_Blacklist.txt', 'w') as f:
        natl_blacklist = ["'" + str(x) + "'" for x in natl_blacklist]        
        f.write(', '.join(natl_blacklist))
except Exception as e:
    print(e)

try:    
    fn.save_data(df_natl_metrics, df_natl_mk_mag, df_natl_mk_dur, df_natl_mk_intra, dataset_name)
except Exception as e:
    print(f'Error saving data: {e}')


[---Working on ME---]
Total Sites: 68 in the state of ME
Working on ME site 1/68 (01010000)
IGNORED: Site 01010070 is in aquifer blacklist
Working on ne site 3/68 (01010500)
Working on ne site 4/68 (01011000)
Working on ne site 5/68 (01013500)
Working on ne site 6/68 (01014000)
Working on ne site 7/68 (01015800)
Working on ne site 8/68 (01017000)
IGNORED: Site 01017290 is in aquifer blacklist
IGNORED: Site 01017550 is in aquifer blacklist
IGNORED: Site 01017960 is in aquifer blacklist
IGNORED: Site 01018009 is in aquifer blacklist
IGNORED: Site 01018035 is in aquifer blacklist
Working on ne site 14/68 (01019000)
Working on ne site 15/68 (01021000)
IGNORED: Site 01021470 is in aquifer blacklist
IGNORED: Site 01021480 is in aquifer blacklist
Working on ne site 18/68 (01022500)
IGNORED: Site 01022840 is in aquifer blacklist
IGNORED: Site 01027200 is in aquifer blacklist
IGNORED: Site 01027240 is in aquifer blacklist
IGNORED: Site 01029200 is in aquifer blacklist
Working on ne site 23/68 (

## EXTRA: Nationwide Validity Dataset
Generates a dataset for every water gauge with 00060_Mean data in the contiguous US indicating gauge validity for this study

In [4]:
date_range = 30
shapefile_path = 'ShapeFiles/Lower48/lower48.shp'
test_limit = 50

In [5]:
df_natl_validity = pd.DataFrame()

for i, state in enumerate(fn.STATE_LIST):
    if i >= test_limit: break
    print(f'[---Working on {state}---]')
    state_uri = fn.create_state_uri(state, fn.PARAM_CODE)
    df_state_sites = filter_state_site(shapefile_path, state_uri)
    print(f'Total Sites: {len(df_state_sites)} in the state of {state}')
    
    ignored_count = 0
    for loc, row in df_state_sites.iterrows():
        df = nwis.get_record(sites=row['site_no'], service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end=fn.DEFAULT_END)
        df = df.reset_index()
        
        if '00060_Mean' not in df.columns:
            ignored_count += 1
            continue
        
        start = df['datetime'].min().date()
        end = df['datetime'].max().date()
        range = round((end - start).days / 365.25, 1)
        
        valid_range = range > date_range
        
        date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * date_range)
        df = df[df['datetime'].dt.date >= date_threshold]
        missing = fn.validate(df, date_threshold, fn.DEFAULT_END)
        valid = missing < fn.MAX_MISSING_THRESHOLD
        
        data = {'site_no': row['site_no'], 'state': state, 'data_range': valid_range, 'data_cont': valid, 'start': start, 'end': end, 
                'total_record': range, 'missing': missing, 'dec_lat_va': row['dec_lat_va'], 'dec_long_va': row['dec_long_va']}
        
        df_natl_validity = pd.concat([df_natl_validity, pd.DataFrame(data, index=['0'])], axis=0, ignore_index=True)
        
    print(f'Ignored {ignored_count} sites ({round((ignored_count / len(df_state_sites)) * 100, 2)}%)')        
        
try:
    with pd.ExcelWriter('Prelim_Data/National_Validity_30.xlsx') as writer:
        df_natl_validity.to_excel(writer, sheet_name='national_validity', index=False) 
except Exception as e:
    print(e)

[---Working on AL---]
Total Sites: 173 in the state of AL
Ignored 35 sites (20.23%)
[---Working on AZ---]
Total Sites: 218 in the state of AZ
Ignored 11 sites (5.05%)
[---Working on AR---]
Total Sites: 139 in the state of AR
Ignored 16 sites (11.51%)
[---Working on CA---]
Total Sites: 481 in the state of CA
Ignored 54 sites (11.23%)
[---Working on CO---]
Total Sites: 349 in the state of CO
Ignored 14 sites (4.01%)
[---Working on CT---]
Total Sites: 74 in the state of CT
Ignored 4 sites (5.41%)
[---Working on DE---]
Total Sites: 36 in the state of DE
Ignored 3 sites (8.33%)
[---Working on FL---]
Total Sites: 434 in the state of FL
Ignored 39 sites (8.99%)
[---Working on GA---]
Total Sites: 308 in the state of GA
Ignored 17 sites (5.52%)
[---Working on ID---]
Total Sites: 229 in the state of ID
Ignored 9 sites (3.93%)
[---Working on IL---]
Total Sites: 193 in the state of IL
Ignored 7 sites (3.63%)
[---Working on IN---]
Total Sites: 205 in the state of IN
Ignored 20 sites (9.76%)
[---Wor

## EXTRA: HUC2/HUC4/Aquifer Master Shapefile Creation
The following code combines HUC2, HUC4, and Aquifer shapefiles into master shapefiles, respectively. It's not necessary to run if these files already exist within the ShapeFiles directory

In [14]:
huc2_path = 'ShapeFiles/HUC2'
huc4_path = 'ShapeFiles/HUC4'
aquifer_path = 'ShapeFiles/Aquifers'
aq_ext = '.shp'
huc2 = 'WBDHU2.shp'
huc4 = 'WBDHU4.shp'

create_huc2 = False
create_huc4 = False
create_aquifer = True

huc2_gdf = gpd.GeoDataFrame()
huc4_gdf = gpd.GeoDataFrame()
aquifer_df = gpd.GeoDataFrame()

# HUC2's
if create_huc2:
    for root, dirs, files in os.walk(huc2_path):
        if os.path.basename(root).startswith('WBD_'):
            code = root[-2:]
            if huc2 in files:
                shape = gpd.read_file(os.path.join(root, huc2))
                shape['huc2_code'] = code
                huc2_gdf = pd.concat([huc2_gdf, shape], ignore_index=True)

    huc2_gdf = huc2_gdf.to_crs(4269)
    huc2_gdf.to_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')

# HUC4's    
if create_huc4:
    for root, dirs, files in os.walk(huc4_path):
        if os.path.basename(root).startswith('NHD_H_'):
            code = root[-4:]
            if huc4 in files:
                shape = gpd.read_file(os.path.join(root, huc4))
                shape['huc4_code'] = code
                huc4_gdf = pd.concat([huc4_gdf, shape], ignore_index=True)

    huc4_gdf = huc4_gdf.to_crs(4269)
    huc4_gdf.to_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
    
# Aquifers
if create_aquifer:
    for root, dirs, files in os.walk(aquifer_path):
        for file in files:
            if file.endswith(aq_ext) and dirs:
                if file.startswith('Penn'):
                    shape = gpd.read_file(os.path.join(root, file))
                    shape = shape.iloc[[17]]                    
                else:
                    shape = gpd.read_file(os.path.join(root, file))
                    
                shape = shape.to_crs(4269)
                shape['aq_name'] = file[:-4]
                aquifer_df = pd.concat([aquifer_df, shape], ignore_index=True)
                
    aquifer_df = aquifer_df.to_crs(4269)
    aquifer_df.to_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

## EXTRA: HUC2/4/Aquifer Sorting
Takes a national validity or national metrics dataset (any dataset with dec_lat_va and dec_long_va data), and adds corresponding HUC2, HUC4, and Aquifer columns indicating a water guages presence or lack thereof in each of these boundaries.

<strong>NOTE:</strong> This feature has already been implemented directly into the National Metrics dataset creation. Its only real use is on National Validity datasets

In [2]:
path = 'Prelim_Data/_National_Metrics/'
datasets = ['National_Metrics_30_90.xlsx']
huc2_gdf = gpd.read_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')
huc4_gdf = gpd.read_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
aq_gdf = gpd.read_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

In [3]:
add_data = pd.DataFrame({'site_no': 'NA', 'huc2_code': 'NA', 'huc4_code': 'NA', 'aquifer': 'NA'}, index=['0'])
temp = pd.DataFrame()

for dataset in datasets:
    df = pd.read_excel(f'{path}{dataset}', dtype={'site_no': str})
    sheets = pd.ExcelFile(f'{path}{dataset}')
    sheet = sheets.sheet_names[0]          

    for i, row in df.iterrows():
        point = Point(row['dec_long_va'], row['dec_lat_va'])
        add_data['site_no'] = row['site_no']
        
        # HUC2 assignment
        for j, geo_row in huc2_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["huc2_code"] = geo_row['huc2_code']
                continue
            
        # HUC4 assignment
        for j, geo_row in huc4_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["huc4_code"] = geo_row['huc4_code']
                continue
            
        for j, geo_row in aq_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["aquifer"] = geo_row['aq_name']
                continue
            
        temp = pd.concat([temp, add_data], axis=0, ignore_index=True)
    
    df = pd.merge(df, temp, on='site_no')
    
    with pd.ExcelWriter(f'{path}{dataset}', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet, index=False)      

## EXTRA: National Dataset Splitting by Aquifer
This script splits a national metric dataset or datasets into smaller per-aquifer datasets

In [24]:
natl_path = 'Prelim_Data/_National_Metrics'

# National datasets to split
datasets = ['National_Metrics_30_90.xlsx', 'National_Metrics_30_95.xlsx', 'National_Metrics_50_90.xlsx', 'National_Metrics_50_95.xlsx']
# The datasets to generate from the national dataset
target_aquifers = cl.ALL_AQUIFERS
sheet_names = ['site_metrics', 'mk_magnitude', 'mk_duration', 'mk_intra_annual']

In [18]:
# Iterate over national datasets
for dataset in datasets:
    df_list = []
    for sheet in sheet_names:
        df = pd.read_excel(f'{natl_path}/{dataset}', sheet_name=sheet, dtype=fn.DATASET_DTYPES)
        df_list.append(df)

    # Iterate over target aquifers
    for aquifer in target_aquifers:
        save_path = f"{aquifer.datasets_dir}/{aquifer.name}_{dataset[-10:-8]}_{dataset[-7:-5]}.xlsx"
        for i, df in enumerate(df_list):
            df = df[df['huc4_code'].isin(aquifer.huc4s)]
            
            # Append new sheets to existing file
            if os.path.exists(save_path):
                with pd.ExcelWriter(save_path, mode='a', if_sheet_exists='replace') as writer:
                    df.to_excel(writer, sheet_name=sheet_names[i], index=False)
            # Create file if it doesn't exist (first iteration)                        
            else: 
                with pd.ExcelWriter(save_path, mode='w') as writer:
                    df.to_excel(writer, sheet_name=sheet_names[i], index=False) 